# World Simulation Orchestrator — Explained from Scratch

**Audience:** You have read `hybrid_solver_explained.ipynb` and `hybrid_ivp_solver_explained.ipynb`.  
**Goal:** Understand every line of `simulation.py` — the `World` class that ties everything together.

---

## What does `World` do?

The solver notebooks showed you **how to compute accelerations** (KKT) and **how to step forward in time** (Euler / IVP).  
But a real simulation needs much more:

| Concern | Who handles it |
|---------|---------------|
| Storing bodies, forces, constraints | `World` |
| Clearing & applying forces each step | `World.step()` |
| Calling the solver | `World.step()` / `World.integrate_to()` |
| Detecting ground contact (termination) | `World.step()` |
| Logging every state to CSV | `CSVLogger` via `World` |
| Organizing output files | `World.enable_logging()` |
| Generating plots | `World.save_plots()` |
| Managing multi-component systems | `World.add_system()` |

`World` is the **orchestrator** — it doesn't do physics itself, it coordinates all the pieces.

```
┌─────────────────────────────────────────────────┐
│                    World                        │
│                                                 │
│  bodies[] ─── forces[] ─── constraints[]        │
│       │            │              │              │
│       ▼            ▼              ▼              │
│   ┌──────────────────────────────────┐          │
│   │  step() / integrate_to()         │          │
│   │   1. Clear forces                │          │
│   │   2. Apply all forces            │          │
│   │   3. Call solver (KKT + integrate)│          │
│   │   4. Log state                   │          │
│   │   5. Check termination           │          │
│   └──────────────────────────────────┘          │
│       │                                         │
│       ▼                                         │
│   CSVLogger ──→ simulation.csv                  │
│   save_plots() ──→ trajectory.png, ...          │
└─────────────────────────────────────────────────┘
```


---
# Part 1: Creating a World

The `World.__init__` sets up empty containers and configuration:

```python
def __init__(self, ground_z=0.0, payload_index=0, simulation_name=None, ...):
    self.bodies = []              # rigid bodies
    self.global_forces = []       # forces applied to ALL bodies (e.g. gravity)
    self.interaction_forces = []  # pairwise forces (e.g. springs)
    self.constraints = []         # holonomic constraints (KKT)
    self.systems = []             # component-based systems
    self.t = 0.0                  # simulation clock
    self.ground_z = ground_z      # termination altitude
    self.payload_index = 0        # which body to monitor
    self.force_breakdown = {}     # per-step force diagnostics
```

Let's create one and inspect it.


In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=120)

from aerislab.dynamics.body import RigidBody6DOF
from aerislab.dynamics.forces import Gravity, Drag, Spring
from aerislab.dynamics.constraints import DistanceConstraint
from aerislab.core.solver import HybridSolver
from aerislab.core.simulation import World, EPSILON_GROUND


In [ ]:
# Create a basic World
world = World(ground_z=0.0, payload_index=0)

print(f"Bodies:            {world.bodies}")
print(f"Global forces:     {world.global_forces}")
print(f"Interaction forces:{world.interaction_forces}")
print(f"Constraints:       {world.constraints}")
print(f"Systems:           {world.systems}")
print(f"Time:              {world.t}")
print(f"Ground altitude:   {world.ground_z}")
print(f"Logger:            {world.logger}")
print(f"Output path:       {world.output_path}")
print()
print("→ Empty world, no logging. Ready to be populated.")


---
# Part 2: Populating the World

We'll build the familiar **anchor + payload** example, but this time through the `World` API.

```
  ████ Anchor (1000 kg)   z = 2 m
    │  rod (L = 2 m)
    ● Payload (5 kg)      z = 0 m
    ↓ gravity
```

## 2.1 Adding Bodies

`world.add_body(body)` appends the body and returns its **index** — you'll need this for constraints.


In [ ]:
q_id = np.array([0., 0., 0., 1.])  # identity quaternion

anchor = RigidBody6DOF(
    name="Anchor", mass=1000.0,
    inertia_tensor_body=np.diag([100., 100., 100.]),
    position=np.array([0., 0., 2.]),
    orientation=q_id,
)
payload = RigidBody6DOF(
    name="Payload", mass=5.0,
    inertia_tensor_body=np.diag([0.5, 0.5, 0.5]),
    position=np.array([0., 0., 0.]),
    orientation=q_id,
)

# Add bodies to world — note the returned indices
idx_anchor = world.add_body(anchor)
idx_payload = world.add_body(payload)

print(f"Anchor  added at index {idx_anchor}")
print(f"Payload added at index {idx_payload}")
print(f"world.bodies = {[b.name for b in world.bodies]}")


## 2.2 Adding Forces

`World` distinguishes three categories of forces:

| Category | Method | Example | How applied |
|----------|--------|---------|-------------|
| **Global** | `add_global_force(f)` | Gravity | To **every** body |
| **Per-body** | `body.per_body_forces` | Drag on payload | To **one** body |
| **Interaction** | `add_interaction_force(spring)` | Spring/tether | Between **two** bodies |

Let's add gravity as a global force and drag on the payload.


In [ ]:
# Global force: gravity applies to ALL bodies
gravity = Gravity(g=np.array([0., 0., -9.81]))
world.add_global_force(gravity)

# Per-body force: drag only on the payload
drag = Drag(rho=1.225, Cd=1.0, area=0.5, mode="quadratic")
payload.per_body_forces.append(drag)

print(f"Global forces:     {[type(f).__name__ for f in world.global_forces]}")
print(f"Payload per-body:  {[type(f).__name__ for f in payload.per_body_forces]}")
print(f"Anchor per-body:   {[type(f).__name__ for f in anchor.per_body_forces]}")


## 2.3 Adding Constraints

Constraints are enforced by the KKT solver. They keep bodies rigidly connected.


In [ ]:
# Distance constraint: 2m rod between anchor and payload
rod = DistanceConstraint(
    world_bodies=world.bodies,
    body_i=idx_anchor, body_j=idx_payload,
    attach_i_local=np.zeros(3), attach_j_local=np.zeros(3),
    length=2.0,
)
world.add_constraint(rod)

print(f"Constraints: {len(world.constraints)} ({type(rod).__name__}, L={rod.L} m)")
print(f"Connects body {rod.i} (Anchor) ↔ body {rod.j} (Payload)")


---
# Part 3: `World.step()` — The Heart of the Simulation

This is the most important method. Every call advances time by `dt`. Let's dissect it.

```
step(solver, dt)
│
├─ 1. Clear force accumulators      ← reset F, τ to zero
├─ 1.5 Update component states      ← deployment logic, etc.
├─ 2a. System component forces      ← parachute drag, etc.
├─ 2b. Per-body forces              ← drag on individual bodies
├─ 2c. Global forces                ← gravity on all bodies
├─ 2d. Interaction forces           ← springs between bodies
├─ 3. Record pre-step altitude      ← for ground detection
├─ 4. solver.step()                 ← KKT + Euler integration
├─ 5. Log state to CSV              ← if logger enabled
└─ 6. Check termination             ← ground contact?
```

Let's walk through each sub-step with our example.


## 3.1 — Clear Forces (step 1)

```python
for b in self.bodies:
    b.clear_forces()
self.force_breakdown.clear()
```

Every step starts fresh. Without clearing, forces would **accumulate** across steps!


In [ ]:
# Simulate what clear_forces does
# First, let's put some fake forces on the bodies
payload.apply_force(np.array([100., 0., 0.]))
print(f"Before clear: payload.f = {payload.f}")

payload.clear_forces()
print(f"After clear:  payload.f = {payload.f}")
print("→ Forces reset to zero. Ready for fresh application.")


## 3.2 — Apply Forces (step 2)

Forces are applied in order: system components → per-body → global → interaction.

### Per-body forces
```python
for b in self.bodies:
    for fb in b.per_body_forces:
        fb.apply(b, self.t)
```

### Global forces
```python
for fg in self.global_forces:
    for b in self.bodies:
        fg.apply(b, self.t)  # applies to EVERY body
```

### Interaction forces (springs)
```python
for fpair in self.interaction_forces:
    fpair.apply_pair(self.t)  # computes forces on BOTH bodies
```

Let's see the accumulated forces after application.


In [ ]:
# Clear and apply all forces manually (mimicking step 2)
for b in world.bodies:
    b.clear_forces()

# 2b) Per-body forces
for b in world.bodies:
    for fb in b.per_body_forces:
        fb.apply(b, world.t)

print("After per-body forces:")
print(f"  Anchor:  F = {anchor.f}  (no per-body forces)")
print(f"  Payload: F = {payload.f}  (drag — but v=0, so drag=0)")

# 2c) Global forces (gravity)
for fg in world.global_forces:
    for b in world.bodies:
        fg.apply(b, world.t)

print("\nAfter global forces (gravity):")
print(f"  Anchor:  F = {anchor.f}  (= {anchor.mass} × [0,0,-9.81])")
print(f"  Payload: F = {payload.f}  (= {payload.mass} × [0,0,-9.81])")

# 2d) Interaction forces (none in this example)
for fpair in world.interaction_forces:
    fpair.apply_pair(world.t)

print(f"\nInteraction forces: {len(world.interaction_forces)} (none in this example)")


### Force Breakdown Tracking

The `World` also captures **per-force diagnostics** via `force_breakdown`:

```python
if hasattr(fb, "last_force"):
    self.force_breakdown[f"{b.name}_{type(fb).__name__}"] = fb.last_force.copy()
```

This dictionary maps `"BodyName_ForceType"` → force vector, and is used for detailed force plots.


## 3.3 — Record Pre-Step Altitude & Call Solver (steps 3-4)

```python
# 3) Record altitude before integration (for ground detection)
payload = self.bodies[self.payload_index]
z_pre = payload.p[2]

# 4) Call the solver — this does KKT + Euler integration
solver.step(self.bodies, self.constraints, dt)
self.t += dt
```

The solver handles the physics (see solver notebooks). World just calls it.


In [ ]:
# Let's do a single step and see the result
solver = HybridSolver(alpha=5.0, beta=1.0)
dt = 0.01

z_before = payload.p[2]
print(f"Before step: t={world.t:.4f}s, payload z={z_before:.6f} m")

# Perform one step (this calls assemble_system + solve_kkt + integrate)
stopped = world.step(solver, dt)

z_after = payload.p[2]
print(f"After step:  t={world.t:.4f}s, payload z={z_after:.6f} m")
print(f"Δz = {z_after - z_before:.8f} m")
print(f"Stopped? {stopped}")
print(f"Distance: {np.linalg.norm(anchor.p - payload.p):.6f} m (should be ≈2.0)")


---
# Part 4: Ground Detection & Termination

After each step, World checks: *"Has the payload touched the ground?"*

## Default Termination: Ground Contact

```python
z_post = payload.p[2]
if (z_pre > self.ground_z + EPSILON_GROUND) and (z_post <= self.ground_z):
    # Payload crossed ground level going DOWN
    frac = (z_pre - self.ground_z) / max((z_pre - z_post), EPSILON_GROUND)
    self.t_touchdown = self.t - dt + frac * dt
    stop = True
```

**Key insight:** The payload doesn't land exactly at `ground_z` — it crosses it in one step.  
We use **linear interpolation** to estimate the exact touchdown time:

```
  z_pre ─────●                    Time axis →
              \                   
  ground_z ───\───── · · · · ·   ← frac = (z_pre - ground_z) / (z_pre - z_post)
               \                  
  z_post ───────●                 t_touchdown = t_step_start + frac × dt
```

The tolerance `EPSILON_GROUND = 1e-9` prevents triggering when the payload is already sitting on the ground.


In [ ]:
# Demonstrate the interpolation with a concrete example
ground_z = 0.0
z_pre_example = 0.15    # 15 cm above ground before step
z_post_example = -0.05  # 5 cm below ground after step
dt_example = 0.01
t_step_start = 10.0

frac = (z_pre_example - ground_z) / max((z_pre_example - z_post_example), EPSILON_GROUND)
t_td = t_step_start + frac * dt_example

print("Ground detection interpolation example:")
print(f"  z_pre  = {z_pre_example} m (above ground)")
print(f"  z_post = {z_post_example} m (below ground)")
print(f"  frac   = ({z_pre_example} - {ground_z}) / ({z_pre_example} - {z_post_example}) = {frac:.4f}")
print(f"  t_touchdown = {t_step_start} + {frac:.4f} × {dt_example} = {t_td:.6f} s")
print(f"\n  → Payload was 75% through the step when it hit the ground.")


## Custom Termination Callbacks

You can override the default ground detection:

```python
world.set_termination_callback(
    lambda w: w.bodies[0].p[2] < 500.0  # stop at 500m altitude
)
```

The callback receives the entire `World` object, so you can check any condition.


In [ ]:
# Demonstrate custom termination
# Stop when velocity exceeds a threshold
def stop_at_speed(w):
    v_mag = np.linalg.norm(w.bodies[w.payload_index].v)
    return v_mag > 50.0  # stop if payload faster than 50 m/s

world.set_termination_callback(stop_at_speed)
print(f"Custom termination set: stop when |v| > 50 m/s")
print(f"Current |v| = {np.linalg.norm(payload.v):.4f} m/s")

# Reset to default for next examples
world.termination_callback = None
print("Reset to default ground detection.")


---
# Part 5: `World.run()` — The Complete Fixed-Step Loop

```python
def run(self, solver, duration, dt, log_interval=1.0):
    t_end = self.t + duration
    
    # Log initial state
    if self.logger: self.logger.log(self)
    
    while self.t < t_end:
        if self.step(solver, dt):   # ← returns True when terminated
            break
        # Print progress every log_interval seconds
        ...
    
    # Cleanup
    if self.logger: self.logger.flush()
    if self._auto_save_plots: self.save_plots()
```

It's fundamentally a while-loop around `step()`. Let's run a full simulation.


In [ ]:
# Fresh world for a complete simulation
world2 = World(ground_z=0.0, payload_index=1)

anchor2 = RigidBody6DOF(
    "Anchor", 1000.0, np.diag([100., 100., 100.]),
    np.array([0., 0., 102.]), q_id  # 102m altitude
)
payload2 = RigidBody6DOF(
    "Payload", 5.0, np.diag([0.5, 0.5, 0.5]),
    np.array([0., 0., 100.]), q_id  # 100m altitude
)

world2.add_body(anchor2)
world2.add_body(payload2)
world2.add_global_force(Gravity(np.array([0., 0., -9.81])))
world2.add_constraint(DistanceConstraint(
    world2.bodies, 0, 1, np.zeros(3), np.zeros(3), 2.0
))

solver2 = HybridSolver(alpha=5.0, beta=1.0)

print("Initial state:")
print(f"  Anchor:  z = {anchor2.p[2]:.1f} m")
print(f"  Payload: z = {payload2.p[2]:.1f} m")
print(f"  Ground:  z = {world2.ground_z} m")
print(f"\nRunning simulation...\n")

world2.run(solver2, duration=10.0, dt=0.01, log_interval=1.0)

print(f"\nFinal state:")
print(f"  Anchor:  z = {anchor2.p[2]:.4f} m")
print(f"  Payload: z = {payload2.p[2]:.4f} m")
print(f"  Touchdown: {world2.t_touchdown}")
print(f"  Distance: {np.linalg.norm(anchor2.p - payload2.p):.6f} m")


### What just happened?

1. Both bodies started at z≈100m, connected by a 2m rod
2. Gravity pulled them both down at ≈9.81 m/s²
3. The constraint kept them exactly 2m apart
4. When the payload crossed z=0 (ground), the simulation stopped
5. Linear interpolation gave us the precise touchdown time

Free-fall from 100m: $t = \sqrt{2h/g} = \sqrt{200/9.81} \approx 4.52$ s — check the result above!


---
# Part 6: Logging & Output Organization

## 6.1 `enable_logging()` — Directory Structure

```python
world.enable_logging("my_simulation")
```

Creates:
```
output/
  my_simulation_20260213_101530/
    logs/
      simulation.csv     ← all state data
    plots/
      payload_trajectory_3d.png
      payload_velocity_acceleration.png
      ...
```

The timestamp prevents overwriting previous runs.

## 6.2 `CSVLogger` — What Gets Logged

Each call to `logger.log(world)` writes one row with:

| Column | Example | Description |
|--------|---------|-------------|
| `t` | 0.0100000000 | Simulation time |
| `Payload.p_x` | 0.0000000000 | Position x |
| `Payload.p_y` | 0.0000000000 | Position y |
| `Payload.p_z` | 99.9995095000 | Position z |
| `Payload.v_x` | ... | Velocity x |
| ... | ... | q, w, f, tau for each body |

## 6.3 `World.with_logging()` — Convenience Factory

```python
world = World.with_logging(
    name="drop_test_01",
    ground_z=0.0,
    auto_save_plots=True  # plots generated automatically
)
```

This is a `@classmethod` that creates a World with logging pre-configured.


In [ ]:
# Demonstrate logging (without running a full sim)
import tempfile, os

with tempfile.TemporaryDirectory() as tmpdir:
    world_log = World(
        ground_z=0.0, payload_index=1,
        output_dir=tmpdir, auto_timestamp=False
    )
    world_log.enable_logging("demo_sim")
    
    # Add bodies
    a = RigidBody6DOF("Anchor", 1000.0, np.diag([100.,100.,100.]),
                       np.array([0.,0.,2.]), q_id)
    p = RigidBody6DOF("Payload", 5.0, np.diag([0.5,0.5,0.5]),
                       np.array([0.,0.,0.]), q_id)
    world_log.add_body(a)
    world_log.add_body(p)
    
    # Log one state
    world_log.logger.log(world_log)
    world_log.logger.flush()
    
    # Show what was created
    csv_path = world_log.output_path / "logs" / "simulation.csv"
    content = csv_path.read_text()
    lines = content.strip().split('\n')
    
    print(f"CSV path: {csv_path}")
    print(f"\nHeader ({len(lines[0].split(','))} columns):")
    for col in lines[0].split(',')[:10]:
        print(f"  {col}")
    print(f"  ... ({len(lines[0].split(',')) - 10} more columns)")
    print(f"\nData rows: {len(lines) - 1}")
    
    # Directory structure
    print(f"\nDirectory structure:")
    for root, dirs, files in os.walk(world_log.output_path):
        level = root.replace(str(world_log.output_path), '').count(os.sep)
        indent = '  ' * level
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}  {f}")
    
    world_log.disable_logging()


---
# Part 7: `integrate_to()` — Variable-Step Solver

For stiff systems (parachute deployment), the fixed-step solver needs tiny `dt`.  
The IVP solver adapts step size automatically.

```python
from aerislab.core.solver import HybridIVPSolver

solver = HybridIVPSolver(method="Radau", rtol=1e-6, atol=1e-8)
sol = world.integrate_to(solver, t_end=100.0, log_interval=1.0)
```

`integrate_to()` works differently from `run()`:

| Feature | `run()` | `integrate_to()` |
|---------|---------|------------------|
| Solver type | Fixed-step `HybridSolver` | Adaptive `HybridIVPSolver` |
| Step size | You choose `dt` | Algorithm chooses |
| Internal loop | `while self.t < t_end` | `scipy.solve_ivp` |
| Force application | Per-step in `step()` | Inside IVP's `rhs()` function |
| Returns | `None` | `OdeResult` object |

The `integrate_to()` method can also chunk the integration for progress updates.


---
# Part 8: Energy Diagnostics

The `get_energy()` method computes kinetic + potential energy for validation.

$$KE = \sum_i \frac{1}{2} m_i |v_i|^2 + \frac{1}{2} \omega_i^T I_i \omega_i$$

$$PE = -\sum_i m_i \cdot g \cdot (p_i - p_{\text{ground}})$$

In a gravity-only system without dissipation, total energy should be conserved.


In [ ]:
# Track energy over a short simulation
world3 = World(ground_z=-1000.0, payload_index=0)  # ground far below
b = RigidBody6DOF("Ball", 10.0, np.diag([1.,1.,1.]),
                   np.array([0.,0.,100.]), q_id)  # 100m up
world3.add_body(b)
world3.add_global_force(Gravity(np.array([0., 0., -9.81])))

solver3 = HybridSolver(alpha=0.0, beta=0.0)  # no constraints

energies = []
times = []
for _ in range(200):
    world3.step(solver3, 0.01)
    e = world3.get_energy()
    energies.append(e)
    times.append(world3.t)

print(f"Energy at t=0.01: KE={energies[0]['kinetic']:.2f} J, PE={energies[0]['potential']:.2f} J, Total={energies[0]['total']:.2f} J")
print(f"Energy at t=2.00: KE={energies[-1]['kinetic']:.2f} J, PE={energies[-1]['potential']:.2f} J, Total={energies[-1]['total']:.2f} J")
print(f"\nEnergy drift: {abs(energies[-1]['total'] - energies[0]['total']):.4f} J")
print("→ Should be ~0 for unconstrained free fall with symplectic Euler.")


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Energy components
axes[0].plot(times, [e['kinetic'] for e in energies], label='Kinetic')
axes[0].plot(times, [e['potential'] for e in energies], label='Potential')
axes[0].plot(times, [e['total'] for e in energies], 'k--', label='Total', linewidth=2)
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Energy [J]')
axes[0].set_title('Energy Conservation (Free Fall)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Total energy zoom
axes[1].plot(times, [e['total'] for e in energies], 'k-')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Total Energy [J]')
axes[1].set_title('Total Energy (Zoomed)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---
# Part 9: System Architecture — `add_system()`

For complex assemblies (parachute + payload + tether), the **component architecture** provides a higher-level API.

```python
from aerislab.components import System, Payload, Parachute

system = System("recovery_system")
system.add_component(Payload(...))
system.add_component(Parachute(...))
system.add_constraint(tether)

world.add_system(system)  # registers ALL bodies + constraints
```

### What `add_system()` does:

```python
def add_system(self, system):
    self.systems.append(system)
    for component in system.components:
        self.add_body(component.body)        # register each body
    for constraint in system.constraints:
        self.add_constraint(constraint)       # register each constraint
```

### How systems integrate with `step()`:

At each step, before solver is called:
1. `system.update_all_states(t, dt)` — updates deployment state, actuation, etc.
2. `system.apply_all_forces(t)` — each component applies its own forces (drag, thrust, etc.)

This separates **domain logic** (parachute deployment) from **simulation mechanics** (KKT solving).


---
# Part 10: Complete Example — Constrained Drop with Logging

Let's put it all together: create a world, populate it, run the simulation, and examine the results.


In [ ]:
import tempfile

# Create world with logging in a temp directory
with tempfile.TemporaryDirectory() as tmpdir:
    world_full = World(
        ground_z=0.0,
        payload_index=1,  # monitor the payload (index 1)
        simulation_name="walkthrough_demo",
        output_dir=tmpdir,
        auto_timestamp=False,
        auto_save_plots=False,
    )
    
    # Bodies
    anchor_full = RigidBody6DOF(
        "Anchor", 1000.0, np.diag([100.,100.,100.]),
        np.array([0., 0., 52.]), q_id
    )
    payload_full = RigidBody6DOF(
        "Payload", 5.0, np.diag([0.5,0.5,0.5]),
        np.array([0., 0., 50.]), q_id
    )
    world_full.add_body(anchor_full)
    world_full.add_body(payload_full)
    
    # Forces
    world_full.add_global_force(Gravity(np.array([0., 0., -9.81])))
    
    # Constraint
    world_full.add_constraint(DistanceConstraint(
        world_full.bodies, 0, 1, np.zeros(3), np.zeros(3), 2.0
    ))
    
    # Run
    solver_full = HybridSolver(alpha=5.0, beta=1.0)
    world_full.run(solver_full, duration=20.0, dt=0.01, log_interval=1.0)
    
    # Read CSV and plot manually
    import pandas as pd
    csv_path = world_full.output_path / "logs" / "simulation.csv"
    df = pd.read_csv(csv_path)
    
    print(f"\nLogged {len(df)} rows, {len(df.columns)} columns")
    print(f"Time range: {df['t'].iloc[0]:.4f} → {df['t'].iloc[-1]:.4f} s")
    print(f"Touchdown time: {world_full.t_touchdown:.4f} s")


In [ ]:
# Read the CSV from above (reusing df)
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 1) Altitude over time
axes[0,0].plot(df['t'], df['Anchor.p_z'], label='Anchor z')
axes[0,0].plot(df['t'], df['Payload.p_z'], label='Payload z')
axes[0,0].axhline(0, color='brown', linestyle='--', alpha=0.5, label='Ground')
axes[0,0].set_xlabel('Time [s]'); axes[0,0].set_ylabel('Altitude [m]')
axes[0,0].set_title('Altitude over Time')
axes[0,0].legend(); axes[0,0].grid(True, alpha=0.3)

# 2) Velocity
axes[0,1].plot(df['t'], df['Payload.v_z'], label='Payload vz')
axes[0,1].set_xlabel('Time [s]'); axes[0,1].set_ylabel('Velocity [m/s]')
axes[0,1].set_title('Payload Vertical Velocity')
axes[0,1].legend(); axes[0,1].grid(True, alpha=0.3)

# 3) Distance constraint
dist = np.sqrt(
    (df['Anchor.p_x'] - df['Payload.p_x'])**2 +
    (df['Anchor.p_y'] - df['Payload.p_y'])**2 +
    (df['Anchor.p_z'] - df['Payload.p_z'])**2
)
axes[1,0].plot(df['t'], dist, 'r-')
axes[1,0].axhline(2.0, color='k', linestyle='--', alpha=0.5, label='Required')
axes[1,0].set_xlabel('Time [s]'); axes[1,0].set_ylabel('Distance [m]')
axes[1,0].set_title('Constraint Satisfaction')
axes[1,0].legend(); axes[1,0].grid(True, alpha=0.3)

# 4) Forces on payload
axes[1,1].plot(df['t'], df['Payload.f_z'], label='Payload Fz')
axes[1,1].axhline(-5*9.81, color='gray', linestyle=':', alpha=0.5, label=f'mg = {-5*9.81:.1f}')
axes[1,1].set_xlabel('Time [s]'); axes[1,1].set_ylabel('Force [N]')
axes[1,1].set_title('Payload Vertical Force')
axes[1,1].legend(); axes[1,1].grid(True, alpha=0.3)

plt.suptitle('Complete Simulation Results', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


---
# Summary: The World Simulation Pipeline

```
┌─────────────────────────────────────────────────────────────────────┐
│                        World (orchestrator)                        │
├──────────────────┬──────────────────────────────────────────────────┤
│  CONFIGURATION   │  EXECUTION                                     │
│                  │                                                 │
│  add_body()      │  run(solver, duration, dt):                     │
│  add_global_     │    log initial state                            │
│    force()       │    while t < t_end:                             │
│  add_interaction │      step(solver, dt):                          │
│    _force()      │        1. clear_forces()                        │
│  add_constraint()│        2. system.update_states()                │
│  add_system()    │        3. system.apply_forces()                 │
│                  │        4. per_body_forces.apply()               │
│  enable_         │        5. global_forces.apply()                 │
│    logging()     │        6. interaction_forces.apply_pair()       │
│  set_termination │        7. solver.step() → KKT + integrate      │
│    _callback()   │        8. logger.log()                          │
│                  │        9. check termination                     │
│                  │    flush logger                                  │
│                  │    auto save plots                               │
│                  │                                                 │
│  ALTERNATIVE:    │  integrate_to(ivp_solver, t_end):               │
│  with_logging()  │    scipy.solve_ivp handles stepping             │
│  (factory)       │    forces applied inside rhs() function         │
│                  │    terminal events for ground detection          │
├──────────────────┴──────────────────────────────────────────────────┤
│  ANALYSIS                                                          │
│    save_plots() → trajectory, velocity, force plots                │
│    get_energy() → KE + PE diagnostic                               │
└────────────────────────────────────────────────────────────────────┘
```

### Key Design Decisions

1. **Separation of concerns**: World orchestrates, Solver computes, Logger records
2. **Force categories**: Global (gravity), per-body (drag), interaction (springs), system-managed (parachute)
3. **Two solver paths**: Fixed-step (`run`) for simplicity, adaptive-step (`integrate_to`) for production
4. **Baumgarte stabilization**: Prevents constraint drift over long simulations
5. **Touchdown interpolation**: Sub-step accuracy for ground contact timing
6. **Component architecture**: `System` groups related bodies/forces for domain modeling
